In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Data_Generation_PP.jl")
include("Estimation_PP.jl")
include("Models_PP.jl")
include("Evaluation_PP.jl")

compute_Profit (generic function with 1 method)

## Functions

In [2]:
function set_Params(N, N_u, K, S_train, S_test, offdiag_sign, max_offdiag, P_bar, RO_coef_all, seed)
    Params = Dict()
    Params["N"] = N
    Params["N_u"] = N_u
    Params["K"] = K
    Params["S_train"] = S_train
    Params["S_test"] = S_test
    # Params["offdiag_sign"] = offdiag_sign
    # Params["max_offdiag"] = max_offdiag
    Params["P_bar"] = P_bar
    # Params["RO_coef_all"] = RO_coef_all
    Params["seed"] = seed
    return Params
end

set_Params (generic function with 1 method)

## Evaluate

#### Baseline Parameters

In [3]:
seed = 1;
N = 3; # num of products
N_u = 1; 
K = 20;
P_dag = repeat(collect(range(0, 30, length=K))', N, 1);
S_train = 200;
S_test = 1000;
P_bar = 30.0;
iterations = 10;
# RO_coef_all = [0.0];
RO_coef_all = [0.0,0.01,0.05,0.1,0.12,0.15,0.2,0.25,0.3];
instances = 1
is_original = false
Random.seed!(seed)
project_dir = "N=$(N)_N_u=$(N_u)_S_train=$(S_train)/"

"N=3_N_u=1_S_train=200/"

In [4]:
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Pricing_Promotion/")
if !isdir(data_dir)
    mkpath(data_dir)
end
data_dir = string(data_dir,project_dir)
println("Data directory: ", data_dir)
Params = set_Params(N, N_u, K, S_train, S_test, "mix", 1.0, P_bar, RO_coef_all, seed);
save(string(data_dir, "Params.jld2"), Params);

Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Pricing_Promotion/N=3_N_u=1_S_train=200/


- Generate Data

In [5]:
Input_Data = Dict()
ins = 1
while ins <= instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    # ****** Data ******
    a_true,b_true,c_true = Generate_Params(N,is_original);
    P_sample, PM_sample, choice_sample = Generate_Data(a_true,b_true,c_true,N, N_u, S_train,P_bar);
    a_hat,b_hat,c_hat = Estimate(N,P_sample, PM_sample, choice_sample);

    w_true = [a_true;b_true;c_true]
    w_hat = [a_hat;b_hat;c_hat]
    if norm(w_hat .- w_true,2) >= 10.0 # avoid extreme estimation case
        continue
    else
        Input_Data["a_true_ins=$ins"] = a_true
        Input_Data["b_true_ins=$ins"] = b_true
        Input_Data["c_true_ins=$ins"] = c_true
        Input_Data["P_sample_ins=$ins"] = P_sample
        Input_Data["PM_sample_ins=$ins"] = PM_sample
        Input_Data["choice_sample_ins=$ins"] = choice_sample
        Input_Data["a_hat_ins=$ins"] = a_hat
        Input_Data["b_hat_ins=$ins"] = b_hat
        Input_Data["c_hat_ins=$ins"] = c_hat
        ins = ins + 1
    end
    println(data_dir_ins)
end
save(string(data_dir,"Input_Data.jld2"),Input_Data)

/Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Pricing_Promotion/N=3_N_u=1_S_train=200/instance=1/


- Run Oracle

In [6]:
function get_input_data_ins(Input_Data,ins)
    a_true = Input_Data["a_true_ins=$ins"]
    b_true = Input_Data["b_true_ins=$ins"]
    c_true = Input_Data["c_true_ins=$ins"]
    # Input_Data["P_sample_ins=$ins"] = P_sample
    # Input_Data["PM_sample_ins=$ins"] = PM_sample
    # Input_Data["choice_sample_ins=$ins"] = choice_sample
    a_hat = Input_Data["a_hat_ins=$ins"]
    b_hat = Input_Data["b_hat_ins=$ins"]
    c_hat = Input_Data["c_hat_ins=$ins"]
    return a_true,b_true,c_true,a_hat,b_hat,c_hat
end

get_input_data_ins (generic function with 1 method)

In [ ]:
Result_True = Dict();
for ins in 1:instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    a_true,b_true,c_true,a_hat,b_hat,c_hat = get_input_data_ins(Input_Data,ins)

    obj_val,Y_val,x_val,solve_time = ETO_PP(N,K,a_true,b_true,c_true,P_dag);
    price,profit = compute_Profit(N,a_true,b_true,c_true,Y_val,x_val,P_dag)
    Result_True["obj_ins=$ins"] = obj_val
    Result_True["price_ins=$ins"] = price
    Result_True["Prom_ins=$ins"] = x_val
    Result_True["time_ins=$ins"] = solve_time
    Result_True["profit_ins=$ins"] = profit
    println(data_dir_ins)
end
save(string(data_dir,"Result_True.jld2"),Result_True) 

- Run ETO

In [ ]:
Result_ETO = Dict();
for ins in 1:instances
    data_dir_ins = string(data_dir, "instance=$ins/")
    a_true,b_true,c_true,a_hat,b_hat,c_hat = get_input_data_ins(Input_Data,ins)

    obj_val,Y_val,x_val,solve_time = ETO_PP(N,K,a_hat,b_hat,c_hat,P_dag);
    price,profit = compute_Profit(N,a_true,b_true,c_true,Y_val,x_val,P_dag)
    Result_ETO["obj_ins=$ins"] = obj_val
    Result_ETO["price_ins=$ins"] = price
    Result_ETO["Prom_ins=$ins"] = x_val
    Result_ETO["time_ins=$ins"] = solve_time
    Result_ETO["profit_ins=$ins"] = profit
    println(data_dir_ins)
end
save(string(data_dir,"Result_ETO.jld2"),Result_ETO) 

In [ ]:
obj_Oracle = [Result_True["obj_ins=$ins"] for ins in 1:instances];
obj_ETO = [Result_ETO["obj_ins=$ins"] for ins in 1:instances];
println("Mean obj_Oracle: ", mean(obj_Oracle));
println("Mean obj_ETO: ", mean(obj_ETO));
println("ETO/Oracle: ", mean(obj_ETO)/mean(obj_Oracle));

In [ ]:
Profit_Oracle = [Result_True["profit_ins=$ins"] for ins in 1:instances];
Profit_ETO = [Result_ETO["profit_ins=$ins"] for ins in 1:instances];
# Rev_RO = Dict();
# for RO_coef in RO_coef_all
#     RST_RO_this = RST_RO["RO_coef=$(RO_coef)"]
#     Rev_RO["RO_coef=$(RO_coef)"] = [RST_RO_this["iter=$(iter)_Rev"] for iter in 1:iterations];
# end

println("Mean Rev_Oracle: ", mean(Profit_Oracle));
println("Mean Rev_ETO: ", mean(Profit_ETO));
println("ETO/Oracle: ", mean(Profit_ETO)/mean(Profit_Oracle));
# Rev_RO_Avg = [mean(Rev_RO["RO_coef=$(RO_coef)"]) for RO_coef in RO_coef_all];
# println("Mean Rev_RO: ", Rev_RO_Avg);

In [ ]:
println("Oracle = ",round.(Profit_Oracle,digits=4))
println("ETO = ",round.(Profit_ETO,digits=4))